# Code for the analyses in the paper "Improving Generalization of Norwegian ASR with Limited Linguistic Resources"

## Imports and loading of datasets

In [1]:
# Imports
import pandas as pd
from jiwer import wer, cer
from scipy.stats import power_divergence
from collections import Counter
from pathlib import Path
import re
import numpy as np

In [2]:
# Make a df of the transcriptions
w_lm = True
filenamepattern = re.compile(r"^((?:nbtale12|nbtale3|npsc|nst|rundkast)(?:_(?:test|validation))*)_*(.*)")
if w_lm:
    filenamepattern = re.compile(r"^((?:nbtale12|nbtale3|npsc|nst|rundkast)(?:_(?:test|validation))*)_*(.*)_w_lm")
dfs = {}
csvs = Path("transcriptions").glob("*.csv")
if w_lm:
    csvs = Path("transcriptions_w_lm").glob("*.csv")

for fn in csvs:
    match = filenamepattern.match(fn.stem)
    dataset = match.group(1)
    asr_model = match.group(2)
    df = pd.read_csv(fn)
    if dataset not in dfs.keys():
        df["dataset"] = dataset
        df[asr_model] = df["transcription"]
        df.drop("transcription", axis=1, inplace=True)
        dfs[dataset] = df
    else:
        dfs[dataset][asr_model] = df["transcription"]
df = pd.concat(dfs.values())

def clean_mult_space(trans):
    """Clean up instances of multiple spaces"""
    if isinstance(trans, str):
        trans = trans.strip()
        trans = re.sub(r"\s\s+", " ", trans)
        trans = trans.replace("\\", " ")
    return trans

# This does not appear to have any effect on WER. However, it removes empty strings
# from lexical analysis   
df["standardized_text"] = df["standardized_text"].apply(lambda x: clean_mult_space(x))
df["stortinget_model"] = df["stortinget_model"].apply(lambda x: clean_mult_space(x))
df["combined_short_model"] = df["combined_short_model"].apply(lambda x: clean_mult_space(x))
df["combined_long_model"] = df["combined_long_model"].apply(lambda x: clean_mult_space(x))
df["rundkast_model"] = df["rundkast_model"].apply(lambda x: clean_mult_space(x))

In [3]:
df.dataset.unique()

array(['npsc_test', 'npsc_validation', 'rundkast_test', 'nst_test',
       'nst_validation', 'nbtale12', 'rundkast_validation', 'nbtale3'],
      dtype=object)

In [4]:
df = df.query("dataset not in ['npsc_validation', 'rundkast_validation', 'nst_validation']")

In [5]:
df = df.drop('Unnamed: 0', axis=1).dropna(subset=["standardized_text"]).query("region != 'unknown'")

In [6]:
round(df.groupby("dataset").duration.sum()/3600, 1)

dataset
nbtale12         15.6
nbtale3          11.6
npsc_test         9.1
nst_test         25.6
rundkast_test     5.8
Name: duration, dtype: float64

In [7]:
# Doing replacements in a separate df so we can see how much
# these transcription differences contribute to the results

replacements = {
    r"(\b)n r k s(\b)": r"\1nrks\2",
    r"(\b)n r k(\b)": r"\1nrk\2",
    r"(\b)u s a s(\b)": r"\1usas\2",
    r"(\b)u s a(\b)": r"\1usa\2",
    r"(\b)a f p(\b)": r"\1afp\2",
    r"(\b)u d i s(\b)": r"\1udis\2",
    r"(\b)u d i(\b)": r"\1udi\2",
    r"(\b)e u s(\b)": r"\1eus\2",
    r"(\b)e u(\b)": r"\1eu\2",
    r"(\b)s a s(\b)": r"\1sas\2",
    r"(\b)s m s(\b)": r"\1sms\2",
    r"(\b)n t n u(\b)": r"\1ntnu\2",
    r"(\b)d n b s(\b)": r"\1dnbs\2",
    r"(\b)d n b(\b)": r"\1dnb\2",
    r"(\b)o s s e s(\b)": r"\1osses\2",
    r"(\b)o s s e(\b)": r"\1osse\2",
    r"(\b)s v s(\b)": r"\1svs\2",
    r"(\b)s v(\b)": r"\1sv\2",
    r"-": "",
    r"(\b)mmm(\b)": r"\1\2",
    r"(\b)eee(\b)": r"\1\2",
    r"(\b)qqq(\b)": r"\1\2",
}
def clean_string(string):
    if type(string) == str:
        for k, v in replacements.items():
            string = re.sub(k, v, string)
        if string == " ":
            string = ""
    return string

df_cleaned = df.copy()
df_cleaned["standardized_text"] = df_cleaned["standardized_text"].apply(lambda x: clean_string(x))
df_cleaned["stortinget_model"] = df_cleaned["stortinget_model"].apply(lambda x:clean_string(x))
df_cleaned["combined_short_model"] = df_cleaned["combined_short_model"].apply(lambda x: clean_string(x))
df_cleaned["combined_long_model"] = df_cleaned["combined_long_model"].apply(lambda x: clean_string(x))
df_cleaned["rundkast_model"] = df_cleaned["rundkast_model"].apply(lambda x: clean_string(x))
df_cleaned = df_cleaned[~df_cleaned.standardized_text.isin([' ', ''])]

In [8]:
df["word_count"] = df.standardized_text.apply(lambda x: len(x.split(" ")))
df_cleaned["word_count"] = df_cleaned.standardized_text.apply(lambda x: len(x.split(" ")))

In [9]:
def compute_score(gold, pred, method=wer):
    """Score a segment according to a scoring method
    (wer or cer)."""
    try:
        if type(pred) != str:
            pred = ""
        if type(gold) != str:
            gold = ""
        return method(gold, pred)*100
    except ValueError:
        return 100


In [10]:
# Get number of word and character errors per segments
# This makes it easier to calculate WER and CER independent
# of segment length later
df["rundkast_num_word_errors"] = df.apply(lambda row: compute_score(row["standardized_text"], row["rundkast_model"])*row["word_count"], axis=1)
df["stortinget_num_word_errors"] = df.apply(lambda row: compute_score(row["standardized_text"], row["stortinget_model"])*row["word_count"], axis=1)
df["combined_short_num_word_errors"] = df.apply(lambda row: compute_score(row["standardized_text"], row["combined_short_model"])*row["word_count"], axis=1)
df["combined_long_num_word_errors"] = df.apply(lambda row: compute_score(row["standardized_text"], row["combined_long_model"])*row["word_count"], axis=1)
df_native = df.query("region != 'foreign'")
df_foreign = df.query("region == 'foreign'")

df_cleaned["rundkast_num_word_errors"] = df_cleaned.apply(lambda row: compute_score(row["standardized_text"], row["rundkast_model"])*row["word_count"], axis=1)
df_cleaned["stortinget_num_word_errors"] = df_cleaned.apply(lambda row: compute_score(row["standardized_text"], row["stortinget_model"])*row["word_count"], axis=1)
df_cleaned["combined_short_num_word_errors"] = df_cleaned.apply(lambda row: compute_score(row["standardized_text"], row["combined_short_model"])*row["word_count"], axis=1)
df_cleaned["combined_long_num_word_errors"] = df_cleaned.apply(lambda row: compute_score(row["standardized_text"], row["combined_long_model"])*row["word_count"], axis=1)
df_cleaned_native = df_cleaned.query("region != 'foreign'")
df_cleaned_foreign = df_cleaned.query("region == 'foreign'")

## Calculation functions

In [11]:
def get_wer(df, num_error_col):
    return round(df[num_error_col].sum()/df["word_count"].sum(), 1)

In [12]:
def relative_change(initial_val, final_val):
    return round((final_val-initial_val)/initial_val*100, 1)

In [13]:
def get_score_by_column(df, gb_col, stat_col, count_col):
    """group by gb_col in ds and calculate error rate given a stat_col with segmentwise number of errors"""
    return round(df.groupby(gb_col)[stat_col].sum()/df.groupby(gb_col)[count_col].sum(), 1)

In [14]:
def make_grouped_df(df, gb_col):
    """Pass a groubby column and create a df with scoring and relative change for different datasets
    and dataset combinations"""
    grouped_df = pd.DataFrame(get_score_by_column(df, gb_col, "rundkast_num_word_errors", "word_count"))
    grouped_df.columns = ["rundkast_wer"]
    grouped_df["stortinget_wer"] = get_score_by_column(df, gb_col, "stortinget_num_word_errors", "word_count")
    grouped_df["combined_short_wer"] = get_score_by_column(df, gb_col, "combined_short_num_word_errors", "word_count")
    grouped_df["combined_long_wer"] = get_score_by_column(df, gb_col, "combined_long_num_word_errors", "word_count")
    grouped_df["relative_change_rundkast_to_combined_short_wer"] = grouped_df.apply(lambda row: relative_change(row.rundkast_wer, row.combined_short_wer), axis=1)
    grouped_df["relative_change_stortinget_to_combined_short_wer"] = grouped_df.apply(lambda row: relative_change(row.stortinget_wer, row.combined_short_wer), axis=1)
    grouped_df["relative_change_rundkast_to_combined_long_wer"] = grouped_df.apply(lambda row: relative_change(row.rundkast_wer, row.combined_long_wer), axis=1)
    grouped_df["relative_change_stortinget_to_combined_long_wer"] = grouped_df.apply(lambda row: relative_change(row.stortinget_wer, row.combined_long_wer), axis=1)
    grouped_df["relative_change_combined_short_to_combined_long_wer"] = grouped_df.apply(lambda row: relative_change(row.combined_short_wer, row.combined_long_wer), axis=1)

    return grouped_df

In [15]:
def make_grouped_wer_plot(results_df, x_label):
    dataset_plot = results_df[["rundkast_wer", "stortinget_wer", "combined_short_wer", "combined_long_wer"]].plot(kind='bar', figsize=(10,10))
    dataset_plot.set_xlabel(x_label)
    dataset_plot.set_ylabel("wer")

## Scoring on the corpus as a whole

In [16]:
def get_corpus_stats(df):
    return {
    "wer_rundkast": get_wer(df, "rundkast_num_word_errors"),
    "wer_stortinget" : get_wer(df, "stortinget_num_word_errors"),
    "wer_combined_short": get_wer(df, "combined_short_num_word_errors"),
    "wer_combined_long": get_wer(df, "combined_long_num_word_errors"),
    }

def print_corpus_stats(stats_dict):
    print(f"WER Rundkast: {stats_dict['wer_rundkast']}")
    print(f"WER Stortinget: {stats_dict['wer_stortinget']}")
    print(f"WER combined_short: {stats_dict['wer_combined_short']}")
    print(f"WER combined_long: {stats_dict['wer_combined_long']}")
    print(f"Relative WER change from Rundkast to combined_short: {relative_change(stats_dict['wer_rundkast'], stats_dict['wer_combined_short'])}")
    print(f"Relative WER change from Rundkast to combined_long: {relative_change(stats_dict['wer_rundkast'], stats_dict['wer_combined_long'])}")
    print(f"Relative WER change from Stortinget to combined_short: {relative_change(stats_dict['wer_stortinget'], stats_dict['wer_combined_short'])}")
    print(f"Relative WER change from Stortinget to combined_long: {relative_change(stats_dict['wer_stortinget'], stats_dict['wer_combined_long'])}")
    print(f"Relative WER change from combined_short to combined_long: {relative_change(stats_dict['wer_combined_short'], stats_dict['wer_combined_long'])}")

In [17]:
corpus_stats_native = get_corpus_stats(df_native)
corpus_stats_foreign = get_corpus_stats(df_foreign)
corpus_stats_cleaned_native = get_corpus_stats(df_cleaned_native)
corpus_stats_cleaned_foreign = get_corpus_stats(df_cleaned_foreign)
corpus_stats_no_test = get_corpus_stats(df_native.query("dataset not in ['npsc_test', 'rundkast_test']"))

### Scoring -- native speakers

In [18]:
print_corpus_stats(corpus_stats_native)

WER Rundkast: 14.0
WER Stortinget: 13.5
WER combined_short: 11.7
WER combined_long: 11.0
Relative WER change from Rundkast to combined_short: -16.4
Relative WER change from Rundkast to combined_long: -21.4
Relative WER change from Stortinget to combined_short: -13.3
Relative WER change from Stortinget to combined_long: -18.5
Relative WER change from combined_short to combined_long: -6.0


In [19]:
print_corpus_stats(corpus_stats_cleaned_native)

WER Rundkast: 13.0
WER Stortinget: 12.4
WER combined_short: 10.8
WER combined_long: 10.1
Relative WER change from Rundkast to combined_short: -16.9
Relative WER change from Rundkast to combined_long: -22.3
Relative WER change from Stortinget to combined_short: -12.9
Relative WER change from Stortinget to combined_long: -18.5
Relative WER change from combined_short to combined_long: -6.5


### Scoring -- non-native speakers

In [20]:
print_corpus_stats(corpus_stats_foreign)

WER Rundkast: 28.8
WER Stortinget: 28.9
WER combined_short: 26.9
WER combined_long: 26.1
Relative WER change from Rundkast to combined_short: -6.6
Relative WER change from Rundkast to combined_long: -9.4
Relative WER change from Stortinget to combined_short: -6.9
Relative WER change from Stortinget to combined_long: -9.7
Relative WER change from combined_short to combined_long: -3.0


In [21]:
print_corpus_stats(corpus_stats_cleaned_foreign)

WER Rundkast: 27.5
WER Stortinget: 27.6
WER combined_short: 25.4
WER combined_long: 24.8
Relative WER change from Rundkast to combined_short: -7.6
Relative WER change from Rundkast to combined_long: -9.8
Relative WER change from Stortinget to combined_short: -8.0
Relative WER change from Stortinget to combined_long: -10.1
Relative WER change from combined_short to combined_long: -2.4


## Scoring on the individual datasets

In [22]:
dataset_result_native = make_grouped_df(df_native, "dataset")
dataset_result_cleaned_native = make_grouped_df(df_cleaned_native, "dataset")

In [23]:
# Native speakers
dataset_result_native

,rundkast_wer,stortinget_wer,combined_short_wer,combined_long_wer,relative_change_rundkast_to_combined_short_wer,relative_change_stortinget_to_combined_short_wer,relative_change_rundkast_to_combined_long_wer,relative_change_stortinget_to_combined_long_wer,relative_change_combined_short_to_combined_long_wer
dataset,,,,,,,,,
nbtale12,19.5,20.7,19.1,18.0,-2.1,-7.7,-7.7,-13.0,-5.8
nbtale3,23.2,21.5,19.9,19.2,-14.2,-7.4,-17.2,-10.7,-3.5
npsc_test,14.9,8.1,8.3,7.1,-44.3,2.5,-52.3,-12.3,-14.5
nst_test,7.1,7.8,6.3,6.0,-11.3,-19.2,-15.5,-23.1,-4.8
rundkast_test,15.3,20.5,14.6,14.0,-4.6,-28.8,-8.5,-31.7,-4.1


In [24]:
dataset_result_cleaned_native

,rundkast_wer,stortinget_wer,combined_short_wer,combined_long_wer,relative_change_rundkast_to_combined_short_wer,relative_change_stortinget_to_combined_short_wer,relative_change_rundkast_to_combined_long_wer,relative_change_stortinget_to_combined_long_wer,relative_change_combined_short_to_combined_long_wer
dataset,,,,,,,,,
nbtale12,18.5,19.9,18.1,17.0,-2.2,-9.0,-8.1,-14.6,-6.1
nbtale3,21.4,19.5,18.2,17.5,-15.0,-6.7,-18.2,-10.3,-3.8
npsc_test,13.8,7.4,7.6,6.2,-44.9,2.7,-55.1,-16.2,-18.4
nst_test,7.1,7.8,6.3,6.0,-11.3,-19.2,-15.5,-23.1,-4.8
rundkast_test,13.2,17.0,12.5,11.9,-5.3,-26.5,-9.8,-30.0,-4.8


### Scoring per dialect region

In [25]:
# Looking at the dialect results with only unplanned speech
region_results_nbtale3 = make_grouped_df(df.query("dataset == 'nbtale3'"), "region")
region_results_nbtale3

,rundkast_wer,stortinget_wer,combined_short_wer,combined_long_wer,relative_change_rundkast_to_combined_short_wer,relative_change_stortinget_to_combined_short_wer,relative_change_rundkast_to_combined_long_wer,relative_change_stortinget_to_combined_long_wer,relative_change_combined_short_to_combined_long_wer
region,,,,,,,,,
east,17.5,17.9,15.7,15.1,-10.3,-12.3,-13.7,-15.6,-3.8
foreign,24.8,25.6,22.6,21.6,-8.9,-11.7,-12.9,-15.6,-4.4
mid,26.5,23.2,22.6,21.8,-14.7,-2.6,-17.7,-6.0,-3.5
north,21.8,20.3,18.6,18.0,-14.7,-8.4,-17.4,-11.3,-3.2
south,20.4,18.7,17.5,17.2,-14.2,-6.4,-15.7,-8.0,-1.7
west,26.1,24.0,22.2,21.1,-14.9,-7.5,-19.2,-12.1,-5.0


## Lexical analysis

In [26]:
def make_freqdist(wordlist):
     return sorted(Counter(wordlist).items(), key=lambda pair: pair[1], reverse=True)

In [27]:
df_native_nbtale_nst = df_native.copy().query("dataset not in ['npsc_test', 'rundkast_test']")
df_native_cleaned_nbtale_nst = df_cleaned_native.copy().query("dataset not in ['npsc_test', 'rundkast_test']")

def get_words_from_col(col):
    sentlist = col.tolist()
    return [x for y in sentlist for x in y.split(" ")]

df_words = df_native_nbtale_nst.dropna(subset=["standardized_text", "rundkast_model", "stortinget_model", "combined_short_model", "combined_long_model"])
reference_wds = get_words_from_col(df_words.standardized_text)
rundkast_wds = get_words_from_col(df_words["rundkast_model"])
stortinget_wds = get_words_from_col(df_words["stortinget_model"])
combined_short_wds = get_words_from_col(df_words["combined_short_model"])
combined_long_wds = get_words_from_col(df_words["combined_long_model"])

In [28]:
#Make frequency word lists (native only)
ref_wdcount = make_freqdist(reference_wds)
rundkast_wdcount = make_freqdist(rundkast_wds)
stortinget_wdcount = make_freqdist(stortinget_wds)
combined_short_wdcount = make_freqdist(combined_short_wds)
combined_long_wdcount = make_freqdist(combined_long_wds)

In [29]:
def make_lexicon_df(list_of_freqdists, list_of_names):
    mydict = {}
    for dist, name in zip(list_of_freqdists, list_of_names):
        for word, n in dist:
            if word not in mydict.keys():
                mydict[word] = {nm: 0 for nm in list_of_names}
            mydict[word][name] += n
    return pd.DataFrame(mydict).transpose()

In [30]:
wordfreq_df = make_lexicon_df([ref_wdcount, rundkast_wdcount, stortinget_wdcount, combined_short_wdcount, combined_long_wdcount], ["reference", "rundkast", "stortinget", "combined_short", "combined_long"])

In [31]:
def make_keyness_df(freq_df, ref_col, compare_col):
    """Keyness computation following this: https://alvinntnu.github.io/NTNU_ENC2036_LECTURES/keyword-analysis.html#computing-keynesss"""
    print()
    keyness_df = freq_df[[ref_col, compare_col]].copy()
    keyness_df.columns = ["reference", "comparison"]
    keyness_df["reference_other"] = keyness_df["reference"].sum()-keyness_df["reference"]
    keyness_df["comparison_other"] = keyness_df["comparison"].sum()-keyness_df["comparison"]
    row_sum_word = keyness_df["reference"] + keyness_df["comparison"]
    row_sum_other = keyness_df["reference_other"] + keyness_df["comparison_other"]
    column_sum_ref = keyness_df["reference"] + keyness_df["reference_other"]
    column_sum_comp = keyness_df["comparison"] + keyness_df["comparison_other"]
    table_sum = row_sum_word + row_sum_other
    keyness_df["reference_exp"] = (row_sum_word * column_sum_ref) / table_sum
    keyness_df["comparison_exp"] = (row_sum_word * column_sum_comp) / table_sum
    keyness_df["reference_other_exp"] = (row_sum_other * column_sum_ref) / table_sum
    keyness_df["comparison_other_exp"] = (row_sum_other * column_sum_comp) / table_sum
    keyness_df["log_likelihood"] = keyness_df.apply(lambda x: power_divergence(f_obs=(x["reference"], x["comparison"], x["reference_other"], x["comparison_other"]), f_exp=(x["reference_exp"], x["comparison_exp"], x["reference_other_exp"], x["comparison_other_exp"]), lambda_=0, ddof=1)[0], axis=1)
    keyness_df["chi_square"] = keyness_df.apply(lambda x: power_divergence(f_obs=(x["reference"], x["comparison"], x["reference_other"], x["comparison_other"]), f_exp=(x["reference_exp"], x["comparison_exp"], x["reference_other_exp"], x["comparison_other_exp"]), lambda_=0, ddof=1)[1], axis=1)
    keyness_df = keyness_df[["reference", "comparison", "log_likelihood", "chi_square"]]
    keyness_df["more_freq_in_comp"] = keyness_df["reference"]/keyness_df["reference"].sum() < keyness_df["comparison"]/keyness_df["comparison"].sum()
    return keyness_df

In [32]:
rundkast_keyness_df = make_keyness_df(wordfreq_df, "reference", "rundkast")
stortinget_keyness_df = make_keyness_df(wordfreq_df, "reference", "stortinget")
combined_short_keyness_df = make_keyness_df(wordfreq_df, "reference", "combined_short")
combined_long_keyness_df = make_keyness_df(wordfreq_df, "reference", "combined_long")

/home/pers/.pyenv/versions/scribe/lib/python3.8/site-packages/scipy/stats/_stats_py.py:6769: RuntimeWarning: invalid value encountered in divide
  terms = 2.0 * special.xlogy(f_obs, f_obs / f_exp)
/home/pers/.pyenv/versions/scribe/lib/python3.8/site-packages/scipy/stats/_stats_py.py:6769: RuntimeWarning: invalid value encountered in divide
  terms = 2.0 * special.xlogy(f_obs, f_obs / f_exp)


/home/pers/.pyenv/versions/scribe/lib/python3.8/site-packages/scipy/stats/_stats_py.py:6769: RuntimeWarning: invalid value encountered in divide
  terms = 2.0 * special.xlogy(f_obs, f_obs / f_exp)
/home/pers/.pyenv/versions/scribe/lib/python3.8/site-packages/scipy/stats/_stats_py.py:6769: RuntimeWarning: invalid value encountered in divide
  terms = 2.0 * special.xlogy(f_obs, f_obs / f_exp)


/home/pers/.pyenv/versions/scribe/lib/python3.8/site-packages/scipy/stats/_stats_py.py:6769: RuntimeWarning: invalid value encountered in divide
  terms = 2.0 * special.xlogy(f_obs, f_obs / f_exp)
/home/pers/.pyenv/versions/scribe/lib/python3.8/site-packages/scipy/stats/_stats_py.py:6769: RuntimeWarning: invalid value encountered in divide
  terms = 2.0 * special.xlogy(f_obs, f_obs / f_exp)


/home/pers/.pyenv/versions/scribe/lib/python3.8/site-packages/scipy/stats/_stats_py.py:6769: RuntimeWarning: invalid value encountered in divide
  terms = 2.0 * special.xlogy(f_obs, f_obs / f_exp)
/home/pers/.pyenv/versions/scribe/lib/python3.8/site-packages/scipy/stats/_stats_py.py:6769: RuntimeWarning: invalid value encountered in divide
  terms = 2.0 * special.xlogy(f_obs, f_obs / f_exp)


In [33]:
def highlight_rows(row):
    if row["more_freq_in_comp"]:
        color = '#fa0000' # Red
    else:
        color = '#1F18C0' # Blue
    return ['background-color: {}'.format(color) for r in row]


The log-likelihood score indicates how typical or atypical a term is in the compared corpus relative to the target corpus. Words with higher numbers are more typical/atypical for the compared corpus, while words with low numbers are more similar to the compared corpus. Since both typical and atypical words get high numbers, it makes sense to look at the frequency in both corpora too. 

In [34]:
# Stortinget relative to ground truth
# Red = more frequent in Stortinget

pd.set_option('display.max_rows', 100)
stortinget_keyness_df.sort_values(by="log_likelihood", ascending=False).head(100).style.apply(highlight_rows, axis=1)

,reference,comparison,log_likelihood,chi_square,more_freq_in_comp
,0,3271,4514.052777,0.000000,True
mmm,994,1,1377.508863,0.000000,False
én,272,0,380.533342,0.000000,False
eee,2833,1595,368.955900,0.000000,False
ipod-,237,0,331.552450,0.000000,False
apples,241,1,325.549573,0.000000,False
itunes-politikk,232,0,324.555547,0.000000,False
appels,0,225,309.272401,0.000000,True
hestemaur,240,9,270.675423,0.000000,False
ipod,6,213,246.158029,0.000000,True


In [35]:
# Rundkast relative to ground truth
# Red = more frequent in Rundkast

pd.set_option('display.max_rows', 100)
rundkast_keyness_df.sort_values(by="log_likelihood", ascending=False).head(100).style.apply(highlight_rows, axis=1)

,reference,comparison,log_likelihood,chi_square,more_freq_in_comp
,0,1362,1881.726929,0.000000,True
mmm,994,5,1330.858858,0.000000,False
én,272,0,379.162336,0.000000,False
ipod-,237,0,330.357898,0.000000,False
itunes-politikk,232,0,323.386201,0.000000,False
apples,241,3,307.715813,0.000000,False
appels,0,197,271.758990,0.000000,True
ipod,6,196,224.717751,0.000000,True
politikk,17,229,215.942202,0.000000,True
e,16,191,173.110803,0.000000,True


In [36]:
combined_short_keyness_df.sort_values(by="log_likelihood", ascending=False).head(100).style.apply(highlight_rows, axis=1)

,reference,comparison,log_likelihood,chi_square,more_freq_in_comp
,0,983,1354.752221,0.000000,True
mmm,994,5,1333.571673,0.000000,False
én,272,0,379.907898,0.000000,False
ipod-,237,0,331.007503,0.000000,False
itunes-politikk,232,0,324.022099,0.000000,False
apples,241,2,316.163834,0.000000,False
appels,0,210,289.126336,0.000000,True
ipod,6,216,250.594747,0.000000,True
ee,0,164,225.780362,0.000000,True
politikk,17,228,214.132563,0.000000,True


In [37]:
combined_long_keyness_df.sort_values(by="log_likelihood", ascending=False).head(100).style.apply(highlight_rows, axis=1)

,reference,comparison,log_likelihood,chi_square,more_freq_in_comp
,0,1713,2358.040104,0.000000,True
mmm,994,6,1327.464959,0.000000,False
én,272,0,380.713455,0.000000,False
apples,241,0,337.309620,0.000000,False
qqqq,0,242,332.488689,0.000000,True
ipod-,237,0,331.709382,0.000000,False
itunes-politikk,232,0,324.709167,0.000000,False
appels,0,194,266.523881,0.000000,True
ipod,6,227,264.508543,0.000000,True
politikk,17,200,179.324126,0.000000,True


## Hapax analysis

In [38]:
hap_ref = wordfreq_df.query('reference == 1')
hap_rundkast = wordfreq_df.query('rundkast == 1')
hap_stortinget = wordfreq_df.query('stortinget == 1')
hap_combined_short = wordfreq_df.query('combined_short == 1')
hap_combined_long = wordfreq_df.query('combined_long == 1')

In [39]:
print(f"number of hapaces in ground truth: {hap_ref.reference.count()}")
print(f"number of hapaces in Rundkast: {hap_rundkast.reference.count()}")
print(f"number of hapaces in Stortinget: {hap_stortinget.reference.count()}")
print(f"number of hapaces in Combined Short: {hap_combined_short.reference.count()}")
print(f"number of hapaces in Combined Long: {hap_combined_long.reference.count()}")

number of hapaces in ground truth: 9672
number of hapaces in Rundkast: 13125
number of hapaces in Stortinget: 13576
number of hapaces in Combined Short: 12076
number of hapaces in Combined Long: 12065


In [40]:
hap_rundkast.query("reference == 0").sample(100, random_state=1)

,reference,rundkast,stortinget,combined_short,combined_long
konsumentene,0,1,3,1,1
ruinheuger,0,1,0,0,0
lahei,0,1,0,0,1
besunne,0,1,1,0,1
hjemmepese,0,1,0,0,0
ply,0,1,0,0,0
langtidsledig,0,1,0,0,0
unyancupqqq,0,1,0,0,0
stavkyrkje,0,1,0,0,0
omtregnt,0,1,0,0,1


In [41]:
hap_stortinget.query("reference == 0").sample(100, random_state=1)

,reference,rundkast,stortinget,combined_short,combined_long
godtland,0,0,1,0,0
utvise,0,0,1,0,0
skriftsørrelsen,0,0,1,0,0
porøst,0,0,1,0,0
manoronsyre,0,1,1,1,1
initiativet,0,1,1,1,1
sutte,0,0,1,1,0
druavling,0,1,1,0,1
anslå,0,0,1,0,0
hasiende,0,0,1,0,0


In [42]:
hap_combined_short.query("reference == 0").sample(100, random_state=1)

,reference,rundkast,stortinget,combined_short,combined_long
kabare,0,1,0,1,1
ratene,0,1,1,1,1
spissformuleringen,0,1,1,1,1
paivi,0,0,1,1,0
helger,0,1,0,1,1
eis,0,0,0,1,0
paulsen,0,1,0,1,1
orgler,0,1,0,1,0
kongle,0,0,0,1,1
selgere,0,1,0,1,0


In [43]:
hap_combined_long.query("reference == 0").sample(100, random_state=1)

,reference,rundkast,stortinget,combined_short,combined_long
lines,0,0,2,0,1
utvidelsen,0,1,1,1,1
studium,0,1,0,1,1
ittekvert,0,0,0,0,1
gateliv,0,1,0,0,1
overlegende,0,1,0,1,1
chuffer,0,0,0,0,1
bokholder,0,0,0,0,1
verktøktsfelt,0,0,0,1,1
nelinjen,0,1,0,0,1
